# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_consult').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
data = spark.read.csv("cruise_ship_info.csv",inferSchema=True,header=True)

In [5]:
data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [6]:
from pyspark.ml.feature import StringIndexer

In [7]:
indexer = StringIndexer(inputCol="Cruise_line",outputCol="Cruise_line_idx")

In [8]:
indexed = indexer.fit(data).transform(data)

In [9]:
indexed.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_idx|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|            1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|            1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|            1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|

In [11]:
train_data,test_data = indexed.randomSplit([0.7,0.3])

In [13]:
train_data.count()

111

In [14]:
test_data.count()

47

In [15]:
train_data.describe().show()

+-------+---------+-----------+-----------------+------------------+------------------+------------------+-----------------+-----------------+------------------+-----------------+
|summary|Ship_name|Cruise_line|              Age|           Tonnage|        passengers|            length|           cabins|passenger_density|              crew|  Cruise_line_idx|
+-------+---------+-----------+-----------------+------------------+------------------+------------------+-----------------+-----------------+------------------+-----------------+
|  count|      111|        111|              111|               111|               111|               111|              111|              111|               111|              111|
|   mean| Infinity|       null|15.18018018018018|   73.099972972973|18.706576576576587| 8.198828828828827|8.953783783783791|40.71963963963963| 7.804144144144149|4.666666666666667|
| stddev|      NaN|       null|6.785275767422221|37.243790196111746| 9.529444369521762|1.76690265140

In [18]:
train_data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_idx']

In [20]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [21]:
assembler = VectorAssembler(inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
'Cruise_line_idx'],outputCol='features')

In [22]:
output = assembler.transform(indexed)

In [25]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[6.0,30.276999999...|
|[6.0,30.276999999...|
|[26.0,47.262,14.8...|
|[11.0,110.0,29.74...|
|[17.0,101.353,26....|
|[22.0,70.367,20.5...|
|[15.0,70.367,20.5...|
|[23.0,70.367,20.5...|
|[19.0,70.367,20.5...|
|[6.0,110.23899999...|
|[10.0,110.0,29.74...|
|[28.0,46.052,14.5...|
|[18.0,70.367,20.5...|
|[17.0,70.367,20.5...|
|[11.0,86.0,21.24,...|
|[8.0,110.0,29.74,...|
|[9.0,88.5,21.24,9...|
|[15.0,70.367,20.5...|
|[12.0,88.5,21.24,...|
|[20.0,70.367,20.5...|
+--------------------+
only showing top 20 rows



In [26]:
output.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_idx=16.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0]))]

In [27]:
final_data = output.select('features','crew')

In [28]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [29]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [30]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               116|
|   mean| 7.749137931034492|
| stddev|3.7054191968369596|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [32]:
lr = LinearRegression(labelCol='crew')

In [33]:
lr_model = lr.fit(train_data)

In [34]:
test_results = lr_model.evaluate(test_data)

In [35]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -1.3223090286013477|
|  0.1963722825961387|
|  0.5021510126146094|
| 0.44703530641424294|
| -0.7792450527765098|
| 0.30275343567292445|
| -0.2337647786998831|
|  1.7968683376922794|
| -0.2969929980920831|
|   0.192259517183766|
| -0.4074840879745274|
| -0.2719136783621501|
| -0.6174968568552313|
|-0.29276486194330786|
| -1.2967332945709202|
|  0.1273828783883264|
| -1.4488728750645237|
|  0.8195941229305426|
| -1.3590475643748121|
|    4.20321171339965|
+--------------------+
only showing top 20 rows



In [36]:
test_results.rootMeanSquaredError

0.9599554808129385

In [37]:
test_results.r2

0.8885371637349547

In [39]:
unlabeled_data = test_data.select('features')

In [40]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[5.0,86.0,21.04,9...|
|[5.0,133.5,39.59,...|
|[6.0,110.23899999...|
|[6.0,113.0,37.82,...|
|[7.0,116.0,31.0,9...|
|[8.0,77.499,19.5,...|
|[9.0,59.058,17.0,...|
|[10.0,46.0,7.0,6....|
|[10.0,58.825,15.6...|
|[10.0,77.0,20.16,...|
|[10.0,81.76899999...|
|[10.0,91.62700000...|
|[10.0,110.0,29.74...|
|[11.0,86.0,21.24,...|
|[11.0,138.0,31.14...|
|[12.0,42.0,14.8,7...|
|[13.0,63.0,14.4,7...|
|[13.0,91.0,20.32,...|
|[13.0,138.0,31.14...|
|[15.0,75.33800000...|
+--------------------+
only showing top 20 rows



In [41]:
predictions = lr_model.transform(unlabeled_data)

In [42]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,86.0,21.04,9...| 9.322309028601348|
|[5.0,133.5,39.59,...|12.933627717403862|
|[6.0,110.23899999...| 10.99784898738539|
|[6.0,113.0,37.82,...|11.552964693585757|
|[7.0,116.0,31.0,9...| 12.77924505277651|
|[8.0,77.499,19.5,...| 8.697246564327076|
|[9.0,59.058,17.0,...|7.6337647786998835|
|[10.0,46.0,7.0,6....|2.6731316623077204|
|[10.0,58.825,15.6...| 7.296992998092083|
|[10.0,77.0,20.16,...| 8.807740482816234|
|[10.0,81.76899999...| 8.827484087974527|
|[10.0,91.62700000...|  9.27191367836215|
|[10.0,110.0,29.74...|12.217496856855231|
|[11.0,86.0,21.24,...| 9.592764861943309|
|[11.0,138.0,31.14...| 13.14673329457092|
|[12.0,42.0,14.8,7...| 6.672617121611673|
|[13.0,63.0,14.4,7...| 6.758872875064523|
|[13.0,91.0,20.32,...| 9.170405877069458|
|[13.0,138.0,31.14...|13.119047564374812|
|[15.0,75.33800000...|  8.79678828660035|
+--------------------+------------

In [49]:
print("Coefficients: {} Intercept: {}".format(lr_model.coefficients,lr_model.intercept))

Coefficients: [-0.0138428650981,0.0160469890653,-0.18811495515,0.36420456627,0.900286927291,-0.0120638908524,0.0359491336866] Intercept: -0.25526236830553256


In [43]:
from pyspark.sql.functions import corr

In [44]:
data.select(corr('crew','passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [46]:
data.select(corr('crew','cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

